In [1]:
from IPython.display import HTML, Video

## Детекция

Для детекции я дообучил предобученную модель YOLOv11 medium на видео из сплита train. Обычно, yolo неприхотлива к качеству данных, поэтому я попробовал просто обучить модель на предложенном датасете: я конвертировал его в yolo формат, из гиперпараметров поменял только размер изображения с 640 на 1024, чтобы маленький относительно кадра мяч лучше детектился. Аугментации не использовал.  
Обучал около 10 часов на V100 и метрики на валидации получились очень плохими: recall и precision около 0.3.  

Вторым вариантом было использовать SAHI на дефолтном для yolo размере изображения - 640x640. Так как sahi не умеет нарезать датасет для yolo, я конвертировал его в coco с помощью библиотеки fiftyone, нарезал и конвертировал обратно в yolo. Это очень времязатратное занятие, поэтому ковертировал я только train часть, из него же взял 0.1 для валидации. Метрики на этой валидационной части получились лучше - recall и precision в районе 0.7, но SAHI я по итогу так и не использовал, т.к., пока учился SAHI, заметил особенности разметки - на некоторых кадрах мяч скрывается, например, за ногами у футболистов и заметить его там невозможно, но разметка все равно присутствует. Я запустил первый вариант детектора на нескольких видео и результаты оказались не такими плохими, как метрики. В дальнейшем использовал первый вариант детектора

In [1]:
import cv2
from ultralytics import YOLO
from tqdm import tqdm
import os

model = YOLO("runs/detect/train32/weights/last.pt")

In [12]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def draw_detections(path_images, model, output_name='track.avi', video_size=(1920, 1080)):

    video=cv2.VideoWriter(output_name,  
                          cv2.VideoWriter_fourcc(*"MJPG"), 
                          30, video_size) 

    for image_name in tqdm(sorted(os.listdir(path_images))):
        frame = cv2.imread(os.path.join(path_images, image_name))
        
        results = model.predict(frame, verbose=False)
        
        boxes = results[0].boxes.xywh.tolist()
        
        for box in boxes:
            cv2.rectangle(frame, (int(box[0] - box[2] / 2), int(box[1] - box[3] / 2)), 
                              (int(box[0] + box[2] / 2), int(box[1] + box[3] / 2)),
                              (0, 255, 0), 2)
        
        frame = cv2.resize(frame, video_size)
        video.write(frame)

    video.release()

Видны некоторые проблемы, например, здесь на 14 секунде детектится голова вместо мяча

In [4]:
Video('videos/double_detection.mp4', width=720, height=480) 

In [11]:
draw_detections('data/SoccerNetGS/challenge/SNGS-001/img1', model, output_name='challenge_1.avi', video_size=(1280, 720))

100%|██████████| 750/750 [01:11<00:00, 10.55it/s]


Здесь также на 1 и 7 секундах лицо игрока задетектилось как мяч

In [5]:
Video('videos/challenge_1.mp4', width=720, height=480) 

In [13]:
draw_detections('data/SoccerNetGS/challenge/SNGS-002/img1', model, output_name='challenge_2.avi', video_size=(1280, 720))

100%|██████████| 750/750 [01:08<00:00, 10.99it/s]


Здесь тоже периодически голова детектится как мяч + теряется сам мяч

In [6]:
Video('videos/challenge_2.mp4', width=720, height=480) 

В целом, мяч детектириуется, видно, что много пропусков кадров, но, кажется, это можно вытянуть алгоритмом трекинга. 
Для улучшения детекции нужно попробовать добавить аугментаций по освещению, т.к. некоторые видео из challenge явно светлее train, также я бы доделал вариант с SAHI. 

## Трекинг

Задачей трекинга я прежде глубоко не занимался, поэтому много времени ушло на то чтобы въехать. Я попробовал несколько классических алгоритмов трекинга из opencv, но ни один не показал хорошего результата

Далее я решил попробовать реализовать простой аналог алгоритма SORT для трекинга одного объекта. Я взял реализацию фильтра Калмана (https://machinelearningspace.com/2d-object-tracking-using-kalman-filter/) с уже заполненными матрицами для предсказания движения 2d объекта, закомменировал компоненты отвечающие за ускорение, так как SORT использует модель движения с константной скоростью. Однако, мне не удалось нормально подобрать значения стандартных отклонений для матриц ковариации - если выставить их большими, то фильтр сильно обращает внимание на предыдущие значения положения мяча и скорости и его начинает колбасить при изменениях траектории, если уменьшить параметры, то не понятно, чем это лучше простого предсказания с помощью модели движения. 

In [17]:
import numpy as np

class KalmanFilter(object):
    def __init__(self, dt, u_x,u_y, std_acc, x_std_meas, y_std_meas):
        """
        :param dt: sampling time (time for 1 cycle)
        :param u_x: acceleration in x-direction
        :param u_y: acceleration in y-direction
        :param std_acc: process noise magnitude
        :param x_std_meas: standard deviation of the measurement in x-direction
        :param y_std_meas: standard deviation of the measurement in y-direction
        """

        # Define sampling time
        self.dt = dt

        # Define the  control input variables
        # self.u = np.matrix([[u_x],[u_y]])

        # Intial State
        self.x = np.matrix([[0], [0], [0], [0]])

        # Define the State Transition Matrix A
        self.A = np.matrix([[1, 0, self.dt, 0],
                            [0, 1, 0, self.dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])

        # Define the Control Input Matrix B
        # self.B = np.matrix([[(self.dt**2)/2, 0],
        #                     [0, (self.dt**2)/2],
        #                     [self.dt,0],
        #                     [0,self.dt]])

        # Define Measurement Mapping Matrix
        self.H = np.matrix([[1, 0, 0, 0],
                            [0, 1, 0, 0]])

        #Initial Process Noise Covariance
        self.Q = np.matrix([[(self.dt**4)/4, 0, (self.dt**3)/2, 0],
                            [0, (self.dt**4)/4, 0, (self.dt**3)/2],
                            [(self.dt**3)/2, 0, self.dt**2, 0],
                            [0, (self.dt**3)/2, 0, self.dt**2]]) * std_acc**2

        #Initial Measurement Noise Covariance
        self.R = np.matrix([[x_std_meas**2,0],
                           [0, y_std_meas**2]])

        #Initial Covariance Matrix
        self.P = np.eye(self.A.shape[1])

    def predict(self):
        # Refer to :Eq.(9) and Eq.(10)  in https://machinelearningspace.com/object-tracking-simple-implementation-of-kalman-filter-in-python/?preview_id=1364&preview_nonce=52f6f1262e&preview=true&_thumbnail_id=1795

        # Update time state
        #x_k =Ax_(k-1) + Bu_(k-1)     Eq.(9)
        self.x = np.dot(self.A, self.x) #+ np.dot(self.B, self.u)

        # Calculate error covariance
        # P= A*P*A' + Q               Eq.(10)
        self.P = np.dot(np.dot(self.A, self.P), self.A.T) + self.Q
        return self.x[0:2]
    
    def update(self, z):

        # Refer to :Eq.(11), Eq.(12) and Eq.(13)  in https://machinelearningspace.com/object-tracking-simple-implementation-of-kalman-filter-in-python/?preview_id=1364&preview_nonce=52f6f1262e&preview=true&_thumbnail_id=1795
        # S = H*P*H'+R
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Calculate the Kalman Gain
        # K = P * H'* inv(H*P*H'+R)
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  #Eq.(11)
        
        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))   #Eq.(12)

        I = np.eye(self.H.shape[1])

        # Update error covariance matrix
        self.P = (I - (K * self.H)) * self.P   #Eq.(13)
        return self.x[0:2]

In [18]:
def draw_kalman(path_images, model, output_name='track.avi', video_size=(1920, 1080)):
    video=cv2.VideoWriter(output_name,  
                          cv2.VideoWriter_fourcc(*'MJPG'), 
                          30, video_size) 

    dt = 1 / 30 # 1 секунда / 30 кадров
    u_x, u_y = 0, 0
    std_acc = 0.1
    x_std_meas = 0.01
    y_std_meas = 0.01

    kf = KalmanFilter(dt, u_x, u_y, std_acc, x_std_meas, y_std_meas)

    last_w = 0
    last_h = 0

    for image_name in tqdm(sorted(os.listdir(path_images))):
            frame = cv2.imread(os.path.join(path_images, image_name))

            results = model(frame)

            out = kf.predict()
            x, y = out.A[0], out.A[1]

            if len(results[0].boxes) > 0:
                xy_det = np.expand_dims(results[0].boxes[0].xyxy[0][:2].cpu().numpy(), 1)
                last_w, last_h = results[0].boxes[0].xywh[0][2:].cpu().numpy()

                (x1, y1) = kf.update(xy_det)
            else:
                (x1, y1) = kf.update(np.array([x, y]))

            frame = cv2.circle(frame, (int(x + (last_w / 2)), int(y + (last_h / 2))), radius=10, color=(255, 0, 0), thickness=3)

            for res in results:
                for box in res.boxes:
                    b = box.xyxy[0]  
                    c = box.cls

                    (x_min, y_min, x_max, y_max) = b.tolist()

                    # image = np.array(image)

                    cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)),
                                            (0, 255, 0), 2)
            
            frame = cv2.resize(frame, video_size)
            video.write(frame)

    video.release()

In [19]:
draw_kalman('data/SoccerNetGS/valid/SNGS-021/img1', model, output_name='double_detection_kalman.avi', video_size=(1280, 720))

  0%|          | 0/750 [00:00<?, ?it/s]/tmp/job-2269450/ipykernel_273708/3266603883.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  frame = cv2.circle(frame, (int(x + (last_w / 2)), int(y + (last_h / 2))), radius=10, color=(255, 0, 0), thickness=3)
100%|██████████| 750/750 [00:47<00:00, 15.82it/s]


Синий кружок - предсказание фильтра

In [7]:
Video('videos/double_detection_kalman.mp4', width=720, height=480) 

По итогу я заменил фильтр Калмана простой моделью движения, которая сохраняет последнее положение мяча и вектор скорости, вычисленный по двум последним кадрам. На их основе предсказывается положение мяча в текущем кадре. Если в текущем кадре детектор не нашел мяч, то используется предсказанное положение и последние известные ширина и высота бокса.  

Также я попытался решить проблему с детектированием сразу двух мячей в кадре и скачками детекций при потере мяча. Для этого задается радиус поиска, на основе которого и предсказанного положения мяча рассчитывается область поиска. Все задетекченные боксы за пределами этой области отбрасываются. Если внутри области все же осталось несколько боксов - берется с наибольшим confidence.  
Если в течение нескольких последних кадров модель не может задетектить мяч, то область поиска отключается и мяч ищется по всему кадру. 

In [5]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
def points_in_circle(boxes, center_x, center_y, radius):
    mask = torch.square(boxes[:,0] - torch.full([boxes.shape[0]], center_x, device=device)) + \
           torch.square(boxes[:,1] - torch.full([boxes.shape[0]], center_y, device=device)) <= \
           torch.square(torch.full([boxes.shape[0]], radius, device=device))
    
    return mask

def simple_tracker(path_images, model, output_name='track.avi', radius=150, frames_to_reset=10, video_size=(1920, 1080)):

    video=cv2.VideoWriter(output_name,  
                          cv2.VideoWriter_fourcc(*'MJPG'), 
                          30, video_size) 
    
    frames_counter = frames_to_reset + 1
    
    # последние известные параметры трекинга
    w_last = 0
    h_last = 0
    x_last = 0
    y_last = 0
    v_x = 0 # скорость по x
    v_y = 0 # скорость по y

    for image_name in tqdm(sorted(os.listdir(path_images))):
        frame = cv2.imread(os.path.join(path_images, image_name))

        # Положение мяча на основе положения в предыдущем кадре
        x_pred = x_last + v_x
        y_pred = y_last + v_y
        
        frame = cv2.arrowedLine(frame, (int(x_last), int(y_last)), (int(x_pred), int(y_pred)), (0, 0, 255), 3)
        
        results = model.predict(frame, verbose=False)
        
        # Оставляем только боксы в радиусе предсказанного положения
        mask = torch.full([results[0].boxes.xywh.shape[0]], True)
        if frames_counter <= frames_to_reset:
            mask = points_in_circle(results[0].boxes.xywh, x_pred, y_pred, radius)
        
        boxes = results[0].boxes.xywh[mask]
        confs = results[0].boxes.conf[mask]
        
        if len(boxes) > 0:
            x_det, y_det, w_det, h_det = boxes[confs.argmax()][:4].tolist()
            
            # frame = cv2.circle(frame, (int(simple_x + (last_w / 2)), int(simple_y + (last_h / 2))), radius=10, color=(0, 0, 255), thickness=3)
            
            if frames_counter > frames_to_reset:
                v_x = 0
                v_y = 0
            else:
                v_x = x_det - x_last
                v_y = y_det - y_last
            
            x_last = x_det
            y_last = y_det
            w_last = w_det
            h_last = h_det
            frames_counter = 0

        else:
            frames_counter += 1
            x_last = x_pred
            y_last = y_pred
            

        # отрисовка радиуса        
        #frame = cv2.circle(frame, (int(x_last), int(y_last)), radius=radius, color=(0, 0, 255), thickness=3)
        
        cv2.rectangle(frame, (int(x_last - w_last / 2), int(y_last - h_last / 2)), 
                              (int(x_last + w_last / 2), int(y_last + h_last / 2)),
                              (0, 255, 0), 2)

        frame = cv2.resize(frame, video_size)
        video.write(frame)

    video.release()

In [6]:
simple_tracker('data/SoccerNetGS/valid/SNGS-021/img1', model, output_name='double_detection_tracker.avi', video_size=(1280, 720))

100%|██████████| 750/750 [00:45<00:00, 16.51it/s]


Видно, что голова начинает детектиться позже (15 секунда) - только когда мяч пропадает из кадра

In [8]:
Video('videos/double_detection_tracker.mp4', width=720, height=480) 

In [4]:
simple_tracker('data/SoccerNetGS/challenge/SNGS-001/img1', model, output_name='tracker_challenge_1.avi', video_size=(1280, 720))

100%|██████████| 750/750 [01:03<00:00, 11.75it/s]


Здесь трекер исключает детекцию головы игрока на 1 и 7 секунде (как это было в видео в начале), но дальше отрабатывает очень плохо

In [9]:
Video('videos/tracker_challenge_1.mp4', width=720, height=480) 

На втором видео все так же плохо - на 11 секунде детектор теряет мяч, а трекер по вычисленному вектору скорости уводит бокс куда то вниз

In [11]:
Video('videos/tracker_challenge_2.mp4', width=720, height=480) 

По итогу такой вариант трекера хорошо работает для исключения множественных срабатываний детектора, а также на плавно двигающихся объектах - позволяет заполнить пропуски детекции рассчитанными значениями, но полностью ломается при резком изменении движения - при передачах. С внезапным уводом мяча куда то вниз, как на последнем видео, помог бы правильно настроенный калмановский фильтр или вычисление вектора скользящим средним, но, например, при длительном полете и резкой остановке они, наоборот, будут все портить. Кажется, что в данной задаче подобные решения с фильтрами можно использовать только для примерного расчета положения мяча, а точный трекинг нужно выполнять другими методами.  

Конкретно этот алгоритм можно улучшить обучив нормально детектор, сильный буст, скорее всего, даст SAHI. Также после решения проблемы с резким изменением вектора скорости можно попробовать ввести адаптивный радиус поиска, который будет меняться в зависимости от скорости мяча. Также нужно заметить, что в этой задаче камера двигается и зумится, что также влияет на вычисление вектора скорости мяча, можно попробовать корректировать его через трекинг каких нибудь ключевых точек стадиона. 

Также ради эксперимента я запустил трекер BotSORT из ultralytics. Здесь видны те же проблемы с детекцией голов футболистов, так что обучение модели детекции здесь ключевой момент

In [11]:
import cv2
from ultralytics import YOLO
from tqdm import tqdm
import os


def track_botsort(path_images, model, output_name='track.avi', video_size=(1920, 1080)):
        
        video=cv2.VideoWriter(output_name,  
                        cv2.VideoWriter_fourcc(*'MJPG'), 
                        30, video_size) 

        for image_name in tqdm(sorted(os.listdir(path_images))):
                frame = cv2.imread(os.path.join(path_images, image_name))

                results = model.track(frame, persist=True, tracker="botsort.yaml")

                # Visualize the results on the frame
                annotated_frame = results[0].plot(conf=False)
                
                annotated_frame = cv2.resize(annotated_frame, video_size)
                video.write(annotated_frame)
                
        video.release()

In [12]:
track_botsort('data/SoccerNetGS/challenge/SNGS-001/img1', model, output_name='tracker_botsort_challenge_1.avi', video_size=(1280, 720))

100%|██████████| 750/750 [01:35<00:00,  7.87it/s]


In [13]:
Video('videos/tracker_botsort_challenge_1.mp4', width=720, height=480) 

In [14]:
Video('videos/tracker_botsort_challenge_2.mp4', width=720, height=480) 